In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/final main.csv")
df.shape

(66397, 2)

In [10]:
df.shape

(66397, 2)

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66397 entries, 0 to 66396
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_info  66397 non-null  object
 1   desc          66397 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [13]:

import numpy as np
import re
import itertools
from collections import Counter
import pandas as pd
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


In [14]:
for i in range(0,df.shape[0]):
  df['desc'][i]= clean_str(df['desc'][i])

In [15]:
df.head()

,product_info,desc
0,SEER SECRETS ACTIVE SILVER ION DEODORANT CREAM,1mg body lotion with shea butter vitamin e p 1...
1,SEER SECRETS MILK BODY LOTION SPF 21,1mg body lotion with shea butter vitamin e p 1...
2,VITRO NATURALS ALOE SHEA BODY BUTTER,1mg body lotion with shea butter vitamin e p 1...
3,1MILE HEALTHCARE CLASSIC TONE STETHOSCOPE,1mile healthcare classic tone stethoscope orde...
4,1MILE HEALTHCARE DOCTOR PLUS STETHOSCOPE,1mile healthcare doctor plus stethoscope 606 o...


In [16]:
documents = df['desc'].values.astype("U")

In [17]:

vectorizer = TfidfVectorizer(stop_words='english')
features = vectorizer.fit_transform(documents)

In [11]:
  # #ELBOW METHOD TO FIND K
# cost =[]
# for i in range(1000, 5000,500):
#     KM = KMeans(n_clusters = i, max_iter = 500)
#     KM.fit(features)
#     cost.append(KM.inertia_)     
  
# plt.plot(range(1000,5000), cost, color ='g', linewidth ='3')
# plt.xlabel("Value of K")
# plt.ylabel("Sqaured Error (Cost)")
# plt.show() 
  


In [ ]:
k = 1000
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(features)

In [ ]:
import pickle
pickle.dump(model, open("/content/drive/My Drive/KMEANS CLUTERING with TF-IDF Vectorizer.sav", 'wb'))

In [ ]:
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)


In [18]:
import pickle
model= pickle.load(open("/content/drive/My Drive/KMEANS CLUTERING with TF-IDF Vectorizer.sav", 'rb'))

In [19]:
df['cluster'] = model.labels_

In [ ]:
df['cluster']

0         17
1         17
2         17
3        586
4        586
        ... 
66392    556
66393     83
66394    163
66395    460
66396     83
Name: cluster, Length: 66397, dtype: int32

In [20]:
df.head()

# output the result to a text file.

clusters = df.groupby('cluster')    

In [21]:
k=1000
for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w') # create csv file
    data = clusters.get_group(cluster)[['product_info','desc']] # get title and overview columns
    f.write(data.to_csv(index_label='id')) # set index to id
    f.close()

print("Cluster centroids: \n")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(100):
    print("Cluster %d:" % i)
    for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
        print (' %s' % terms[j])
    print('------------')

Cluster centroids: 

Cluster 0:
 fabbox
 mix
 chaat
 seed
 assist
 loaded
 fiber
 gram
 management
 addition
------------
Cluster 1:
 belladonna
 effect
 convulsion
 treat
 parts
 opium
 potent
 poisoning
 relieve
 dilution
------------
Cluster 2:
 free
 tablet
 shipping
 strip
 10
 tablets
 sold
 taxes
 inclusive
 cashback
------------
Cluster 3:
 magmaxx
 tablet
 ir
 strip
 mrp
 10
 tablets
 mebala
 otc523885products
 gibtulio
------------
Cluster 4:
 14
 lm
 opulus
 viburnum
 price
 globules
 20
 sbl
 gm
 bottle
------------
Cluster 5:
 syrup
 use
 otc296213order
 price
 100
 ml
 bottle
 com
 know
 specifications
------------
Cluster 6:
 mass
 quick
 protein
 size
 mixability
 hardest
 dextrin
 gainers
 amazing
 skimmed
------------
Cluster 7:
 vasu
 aromatics
 oil
 essential
 cream
 dazzle
 rosemary
 price
 cutis
 acnovin
------------
Cluster 8:
 celin
 vitamin
 person
 otc32913celin
 500
 4k
 varies
 depending
 connective
 ascorbic
------------
Cluster 9:
 rosewater
 skin
 natural

In [22]:
def search(str):
  new_doc = [str]
  Y = vectorizer.transform(new_doc)
  prediction = model.predict(Y)
  print(prediction)
  result=pd.read_csv("cluster{}.csv".format(prediction[0]))
  indices = [i for i, s in enumerate(list(result["desc"])) if str in s]
  print(indices)
  if len(indices)==0:
    print(result["product_info"])
  else:
    for i in indices:
      print(result["product_info"][i])

In [24]:
search("tulsi")

[83]
[17, 47, 72, 258, 268, 291, 353, 528, 529, 543, 566, 567, 568, 569, 629, 630, 789, 920, 931, 1220, 1549, 1550, 1556, 1558, 1618, 1628, 2112, 2138, 2145, 2153, 2155, 2398, 2634, 2635, 2636, 2637, 2638, 2639, 2724, 2762, 2763, 2777, 2778, 2779, 2924, 2925, 2943, 2944, 3006, 3008, 3009, 3060, 3092, 3093, 3094, 3113, 3114, 3133, 3224, 3225, 3273, 3274, 3275, 3282, 3283, 3484, 3493, 3589, 3591, 3608, 3609, 3610, 3715, 3722, 3796, 3845, 4265, 4299, 4300, 4301, 4537, 4546, 4775, 5043, 5085, 5222, 5278, 5308, 5467, 5507, 5605, 5607, 5610, 5612, 5772, 5774, 5775, 5806, 5879, 6337, 6364, 6682, 6744, 6826, 7177, 7178, 7491]
AFFLATUS LAUKI RAS
AMOGH AYUSH KWATH POWDER
AVG AYUSH KWATH KADHA TABLET SUGAR
DR VAIDYAS AYUSH KWATH AYURVEDIC KADHA 50GM
EMAMI VASOCARE PETROLEUM JELLY
DIVINECTAR HAIR CONDITIONER TULSI LIME
HEALING PHARMA IGLUCOOL HERBAL PRICKLY HEAT POWDER
MEGHDOOT PALLAVI AYURVEDIC PEYA
MEGHDOOT PRICKLY HEAT TALC POWDER
MINT VEDA PANCH TULSI DROP
NATURE CHAI GINGER TULSI
NATURE AND N